<a href="https://colab.research.google.com/github/huerd/GPA659-E2020/blob/master/_project/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from __future__ import print_function

%tensorflow_version 1.x
import tensorflow as tf
try:
  tf.contrib.eager.enable_eager_execution()
except ValueError:
  pass  # enable_eager_execution errors after its first call

tensor = tf.constant('This is an edit test. Save to Github to commit.')
tensor_value = tensor.numpy()
print(tensor_value.decode())

This is an edit test. Save to Github to commit.
